This is a project implemented by Konstantinos Lampropoulos $$ $$
AM:1115201800092

Imports

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import TensorDataset, DataLoader
from torchmetrics.classification import ConfusionMatrix,F1Score

Load Datasets

In [ ]:
#Load Train set
X_train = np.load('./music_genre_data_di/train/mfccs/X.npy')
Labels_train = np.load('./music_genre_data_di/train/mfccs/labels.npy')

#Load Validation set
X_val = np.load('./music_genre_data_di/val/mfccs/X.npy')
Labels_val = np.load('./music_genre_data_di/val/mfccs/labels.npy')

#Load Test set
X_test = np.load('./music_genre_data_di/test/mfccs/X.npy')
Labels_test = np.load('./music_genre_data_di/test/mfccs/labels.npy')

Encode the labels

In [ ]:
encoder  = LabelEncoder()

Labels_train_encoded = encoder.fit_transform(Labels_train)

Labels_val_encoded = encoder.fit_transform(Labels_val)

Labels_test_encoded = encoder.fit_transform(Labels_test)

Create torch Datasets

In [ ]:
batch_size = 16
#Train set
X_train_tensor = torch.tensor(X_train,dtype=torch.float32)
Labels_train_tensor = torch.tensor(Labels_train_encoded,dtype=torch.long)
Dataset = TensorDataset(X_train_tensor,Labels_train_tensor)
Train_set = DataLoader(Dataset,batch_size=batch_size,shuffle=True)

#Val set
X_val_tensor = torch.tensor(X_val,dtype=torch.float32)
Labels_val_tensor = torch.tensor(Labels_val_encoded,dtype=torch.long)
Dataset = TensorDataset(X_val_tensor,Labels_val_tensor)
Validation_set = DataLoader(Dataset,batch_size=batch_size,shuffle=True)

#Test set
X_test_tensor = torch.tensor(X_test,dtype=torch.float32)
Labels_test_tensor = torch.tensor(Labels_test_encoded,dtype=torch.long)
Dataset = TensorDataset(X_test_tensor,Labels_test_tensor)
Test_set = DataLoader(Dataset,batch_size=batch_size)

Set CPU or GPU

In [ ]:
if torch.cuda.is_available():
   device="cuda"
else:
   device="cpu"
print("Device =",device)

Create Neural Network Class

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(26,128),
            nn.ReLU(),
            nn.Linear(128,32),
            nn.ReLU(),
            nn.Linear(32,4),
        )
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)

In [ ]:
def Train(dataLoader,model,loss_fn,optimizer):
    size = len(dataLoader.dataset)
    for batch, (X,y) in enumerate(dataLoader):
        #Load to GPU
        X = X.to(device)
        y = y.to(device)

        #Prediction and loss
        pred = model(X)
        loss = loss_fn(pred,y)

        #BackPropagation

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [ ]:
def Test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    test_loss,correct,f1 = 0,0,0

    with torch.no_grad():
        for X,y in dataloader:
            #Load to GPU
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            f1_score = F1Score(task='multiclass',num_classes=4,average='macro').to(device)
            f1 += f1_score(pred.argmax(1),y)

    test_loss /= size
    correct /= size
    f1 /= size
    confmat = ConfusionMatrix('multiclass',num_classes=4).to(device)
    confusion_matrix = confmat(pred.argmax(1),y)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f},Avg F1-Score: {f1:>8f}\n")
    print(f"Confusion Matrix \n: {confusion_matrix}\n")

In [ ]:
def Validation(dataloader,model):
    size = len(dataloader.dataset)
    f1 = 0

    with torch.no_grad():
        for X,y in dataloader:
            #Load to GPU
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            f1_score = F1Score(task='multiclass',num_classes=4,average='macro').to(device)
            f1 += f1_score(pred.argmax(1),y)
    f1 /= size
    return f1

In [85]:
loss_fn = nn.CrossEntropyLoss()
best_f1 = 0
learning_rate = 0.002

optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

epochs_count = 30
for t in range(epochs_count):
    print(f"Epoch {t+1}\n-------------------------------")
    Train(Train_set,model,loss_fn,optimizer)
    f1 = Validation(Validation_set,model)
    if f1 >= best_f1:
        best_f1 = f1
        best_model = model
    Test(Test_set,model,loss_fn)
print("Best Model computed by finding the model with the highest f1 score on the validation set\n")
Test(Test_set,best_model,loss_fn)

Epoch 1
-------------------------------
loss: 1.075356  [    0/ 3200]
loss: 1.031522  [ 1600/ 3200]
Test Error: 
 Accuracy: 62.6%, Avg loss: 0.059280,Avg F1-Score: 0.011620

Confusion Matrix 
: tensor([[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 2,  0,  2, 12]], device='cuda:0')

Epoch 2
-------------------------------
loss: 0.926566  [    0/ 3200]
loss: 1.216185  [ 1600/ 3200]
Test Error: 
 Accuracy: 61.1%, Avg loss: 0.061050,Avg F1-Score: 0.011194

Confusion Matrix 
: tensor([[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 2,  0,  3, 11]], device='cuda:0')

Epoch 3
-------------------------------
loss: 0.902760  [    0/ 3200]
loss: 0.963225  [ 1600/ 3200]
Test Error: 
 Accuracy: 63.0%, Avg loss: 0.060047,Avg F1-Score: 0.011879

Confusion Matrix 
: tensor([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [7, 0, 2, 7]], device='cuda:0')

Epoch 4
-------------------------------
loss: 0.860747  [    0/ 3

Με την χρήση CPU για την αξιολόγηση του μοντέλου πετυχαίνουμε 60.6% accuracy,Loss 0.060977 και f1  0.011394 σε 6.5s

Με την χρήση GPU για την αξιολόγιση του μοντέλου πετυχαίνουμε 62.4% accuracy,Loss 0.061011 και f1  0.011620 σε 22.4s